In [25]:
import requests
import json
from datetime import datetime
import os
from dotenv import load_dotenv

load_dotenv()

True

In [26]:
def get_cheap_flights(origin, destination, depart_date, return_date, token):
    url = "https://api.travelpayouts.com/v1/prices/cheap"
    headers = {
        "Content-Type": "application/json",
        "X-Access-Token": os.getenv("TRAVEL_PAYOUTS_API_KEY")
    }
    params = {
        "origin": origin,
        "destination": destination,
        "depart_date": depart_date,
        "return_date": return_date,
    }
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()

        data = response.json()
        if not data.get("success"):
            print("API request failed:", data.get("error", "Unknown error"))
            return None

        flights = data.get("data", {}).get(destination, {})
        if not flights:
            print(f"No flights found from {origin} to {destination} on {depart_date}.")
            return None

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

In [27]:
from langchain.chat_models import init_chat_model

model = init_chat_model("llama3-8b-8192", model_provider="groq")

In [29]:
model.invoke(
    "What are the cheapest flights from New York to Los Angeles on?"
)

AIMessage(content="The cheapest flights from New York to Los Angeles can vary greatly depending on several factors such as travel dates, time of year, demand, and availability. However, I can provide you with some general information and options to help you find the cheapest flights.\n\n**Low-cost carriers:**\n\n1. Spirit Airlines: Spirit often offers the cheapest flights from New York (LGA or JFK) to Los Angeles (LAX) with prices starting from around $150-$200 one-way, depending on the travel dates.\n2. Frontier Airlines: Frontier is another low-cost carrier that operates flights from New York (LGA or JFK) to Los Angeles (LAX) with prices starting from around $180-$250 one-way.\n3. JetBlue: JetBlue is a low-cost carrier that operates flights from New York (JFK) to Los Angeles (LAX) with prices starting from around $200-$300 one-way.\n\n**Budget-friendly airlines:**\n\n1. Delta: Delta often offers affordable flights from New York (JFK) to Los Angeles (LAX) with prices starting from aro

In [30]:
llm_with_tools = model.bind_tools([get_cheap_flights])

In [31]:
llm_with_tools.invoke(
    "Are there any flights to Phuket"
)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '2769a08v1', 'function': {'arguments': '{"depart_date":"today","destination":"Phuket","origin":"your_current_location","return_date":"today","token":"your_api_token"}', 'name': 'get_cheap_flights'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 101, 'prompt_tokens': 880, 'total_tokens': 981, 'completion_time': 0.092932051, 'prompt_time': 0.099082648, 'queue_time': 0.144018, 'total_time': 0.192014699}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_0fb809dba3', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--8baba3c5-214d-4cc3-b2d0-d41aab93af9e-0', tool_calls=[{'name': 'get_cheap_flights', 'args': {'depart_date': 'today', 'destination': 'Phuket', 'origin': 'your_current_location', 'return_date': 'today', 'token': 'your_api_token'}, 'id': '2769a08v1', 'type': 'tool_call'}], usage_metadata={'input_tokens': 880, 'output_tokens': 101, 'total_toke

In [32]:
msg = llm_with_tools.invoke(
    "Are there any flights to Phuket"
)

In [34]:
msg.tool_calls

[{'name': 'get_cheap_flights',
  'args': {'depart_date': '2023-03-15',
   'destination': 'Phuket',
   'origin': 'your_location',
   'return_date': '2023-03-22',
   'token': 'your_flight_token'},
  'id': 'mw2gb4tyt',
  'type': 'tool_call'}]

In [19]:
from typing_extensions import TypedDict, Optional
from langgraph.graph import StateGraph, START, END
from IPython.display import display, Image

In [35]:
#Graph state
class FlightState(TypedDict):
    query: str
    origin: Optional[str]
    destination: Optional[str]
    depart_date: Optional[str]
    return_date: Optional[str]
    api_response: Optional[dict]
    formatted_response: Optional[str]
    stage: str
    missing: list[str]

In [ ]:
#Nodes
def query(state: FlightState):
   '''
   First LLM call to generate the query
   '''
   msg = model.invoke(
    '''
    You are a travel assistant. The user asked: "{query}". Your task is to extract flight search parameters (origin, destination, departure date, return date) from this query. Use IATA codes for cities where possible (e.g., Phuket = HKT). If a parameter is missing or ambiguous, mark it as null and note what’s needed. Assume the current year is {current_year} unless specified. Return a JSON object with the extracted parameters and a list of missing parameters.

    Example input: "Flights from London to Paris in June"
    Example output: {
    "origin": "LON",
    "destination": "PAR",
    "depart_date": "2025-08",
    "return_date": null,
    "missing": ["return_date"]
    }

    User query: "{query}"
    Output: <JSON object>
    '''
   )
   return {"query": msg.content}
